# Recommendations on starting a new business in the City of London

### Table of contents
-   Introduction
-	Data
-	Methodology
-	Analysis
-	Results
-	Discussion
-	Conclusion


### Introduction: Business Problem
London, the capital and financial hub of England has always been one of the most popular expat destinations in the world. Some of the factors that make London a great place to live in include its living standards, rich culture, diversity, benefits, education, career prospects and healthcare, to name a few. There is plenty to see and do for people of all ages and backgrounds but there is a lot more to this unique city than its attractions. Entrepreneurs and budding business people from around the world head to the Big Smoke when in the process of establishing themselves. London has now also become one of the leading hubs for start-ups in Europe.

In this notebook, we will be exploring the different neighbourhoods of London using Foursquare location data to get reviews, ratings, tips and counts, which would give us an insight into the different businesses within London such as bars, restaurants, grocery stores, hotels etc. Once we have enough data available on different businesses, we can then decide what kind of business to establish and where we would make the most returns on investment. The recommendations will be based on the results returned by Foursquare API on major businesses.


### Data
The recommendations will be based on the data returned by four square such as business name, category, review and location. This will help us understand what kind of business will have a great impact and in which location within London.

Following data sources will be needed to extract/generate the required information:
- **Google Maps API using Geocoding** to get the coordinates/latitudes and longitudes of London.
- **Four Square API** to get the name, address, category, street, latitude and longitude of different businesses.
- We will use pandas json_normalize to transform our json data into a dataframe.
- **Folium Maps** to generate maps for the businesses in London and which areas there are located.
